In [1]:
%load_ext autoreload
%autoreload 2

import os
os.environ["WANDB_DISABLED"] = "true"

import sys
sys.path = [
    p for p in sys.path
    if p not in ['/home/jxm3/research/prompting/imodelsX', '/home/jxm3/research/prompting/tree-prompt']
]
sys.path.append('/home/jxm3/research/retrieval/inversion')

In [2]:
import aliases

# inv_trainer = aliases.load_trainer_from_alias("openai_msmarco__msl128__100epoch")
corr_experiment, corr_trainer = aliases.load_experiment_and_trainer_from_alias("gtr_nq__msl32_beta__correct")
inv_trainer = corr_trainer.inversion_trainer
# corr_trainer.precompute_hypotheses()
corr_trainer.model.eval()
print()

[2023-08-29 09:47:58,802] [INFO] [real_accelerator.py:110:get_accelerator] Setting ds_accelerator to cuda (auto detect)

===================================BUG REPORT===================================
Welcome to bitsandbytes. For bug reports, please run

python -m bitsandbytes

 and submit this information together with your error trace to: https://github.com/TimDettmers/bitsandbytes/issues
bin /home/jxm3/.conda/envs/torch/lib/python3.10/site-packages/bitsandbytes/libbitsandbytes_cuda117.so
CUDA SETUP: CUDA runtime path found: /home/jxm3/.conda/envs/torch/lib/libcudart.so.11.0
CUDA SETUP: Highest compute capability among GPUs detected: 8.6
CUDA SETUP: Detected CUDA version 117
CUDA SETUP: Loading binary /home/jxm3/.conda/envs/torch/lib/python3.10/site-packages/bitsandbytes/libbitsandbytes_cuda117.so...
loading alias gtr_nq__msl32_beta__correct from /home/jxm3/research/retrieval/inversion/saves/47d9c149a8e827d0609abbeefdfd89ac...
> checkpoint: /home/jxm3/research/retrieval/inversion/sa

/home/jxm3/.conda/envs/torch/lib/python3.10/site-packages/transformers/models/t5/tokenization_t5_fast.py:155: FutureWarning: This tokenizer was incorrectly instantiated with a model max length of 512 which will be corrected in Transformers v5.
For now, this behavior is kept to avoid breaking backwards compatibility when padding/encoding with `truncation is True`.
- Be aware that you SHOULD NOT rely on t5-base automatically truncating your input to 512 when padding/encoding.
- If you want to encode/pad to sequences longer than 512 you can either instantiate this tokenizer with `model_max_length` or pass `max_length` when encoding/padding.
- To avoid this warning, please instantiate this tokenizer with `model_max_length` set to your preferred value.
  warnings.warn(


Loading datasets with TOKENIZERS_PARALLELISM = False
Renaming keys {'embedding_transform.2.weight', 'embedding_transform.2.bias'} for backward compatibility.
================ Begin trainer sanity check ================
	Input to encode -> Twas brillig, and the slithy toves, Did gyre and gimble in the wabe, All mimsy were the borogoves, And the mome raths outgrabe.
	Decoded output shape ->  torch.Size([1, 33])
	Decoded output -> The mlbies wase wyst bograge; And the sliths and toms wy
================ End trainer sanity check ================
Froze 342572160 params from model type <class 'models.inversion.InversionModel'>
Renaming keys {'embedding_transform.2.weight', 'embedding_transform.2.bias'} for backward compatibility.
Renaming keys {'embedding_transform.2.weight', 'embedding_transform.2.bias'} for backward compatibility.
================ Begin trainer sanity check ================
	Input to encode -> Twas brillig, and the slithy toves, Did gyre and gimble in the wabe, All mimsy w

In [10]:
# load DPR
from sentence_transformers import SentenceTransformer
sentences = ["This is an example sentence", "Each sentence is converted"]

dpr = SentenceTransformer('sentence-transformers/facebook-dpr-ctx_encoder-multiset-base')
embeddings = dpr.encode(sentences)
print(embeddings)

[[-0.11496873  0.06054432 -0.01189671 ... -0.23369355  0.03117392
   0.6132764 ]
 [-0.0163618   0.29337484  0.21915755 ... -0.42278576  0.05200446
   0.51192284]]


In [52]:
# load support set

et = corr_trainer.embedder_tokenizer

from data_helpers import load_beir_datasets
support_set = load_beir_datasets()["msmarco"]

support_set = [t["text"] for t in support_set]
support_set = [et.decode(et(t, max_length=32)["input_ids"], skip_special_tokens=True) for t in support_set]

print(len(support_set))
support_set[0]

loading BEIR dataset: arguana
loading BEIR dataset: climate-fever
loading BEIR dataset: cqadupstack
loading BEIR dataset: dbpedia-entity
loading BEIR dataset: fever
loading BEIR dataset: fiqa
loading BEIR dataset: hotpotqa
loading BEIR dataset: msmarco
loading BEIR dataset: nfcorpus
loading BEIR dataset: nq
loading BEIR dataset: quora
loading BEIR dataset: scidocs
loading BEIR dataset: scifact
loading BEIR dataset: trec-covid
loading BEIR dataset: webis-touche2020
loading BEIR dataset: signal1m
loading BEIR dataset: trec-news
loading BEIR dataset: robust04
loading BEIR dataset: bioasq
100000


'The presence of communication amid scientific minds was equally important to the success of the Manhattan Project as scientific intellect was. The only cloud hanging over the impressive achievement of'

In [53]:
# encode support sets

import torch
support_set_dpr = torch.tensor(dpr.encode(support_set))
print("DPR support set:", support_set_dpr.shape)

support_set_gtr = []
batch_size = 512
idx = 0
while idx < len(support_set):
    tokenized = (
        et(
            support_set[idx : idx + batch_size], 
            max_length=32, 
            truncation=True,
            padding=True, 
            return_tensors='pt'
        ).to(corr_trainer.args.device)
    )
    gtr_embs = corr_trainer.inversion_trainer.call_embedding_model(
        # embedder_input_ids=tokenized.input_ids,
        # embedder_attention_mask=tokenized.attention_mask,
        **tokenized
    )
    support_set_gtr.extend(gtr_embs.cpu().tolist())
    idx += batch_size

support_set_gtr = torch.tensor(support_set_gtr)
print("GTR support set:", support_set_gtr.shape)

DPR support set: torch.Size([100000, 768])
GTR support set: torch.Size([100000, 768])


In [89]:
# embedding text
text = "Mage (foaled April 18, 2020) is my American Thoroughbred racehorse who won the 2023 Kentucky Derby."

def invert_with_support(t: str) -> str:
    # project into support set
    temp = 1 / 50 # 00.
    # gtr_emb = corr_trainer.inversion_trainer.call_embedding_model(
    #     # embedder_input_ids=tokenized.input_ids,
    #     # embedder_attention_mask=tokenized.attention_mask,
    #     **et([text], return_tensors='pt').to(corr_trainer.args.device)
    # ).cpu()
    # gtr_emb = support_set_gtr[None,0]
    # print(gtr_emb.shape)
    # support_set_scores = torch.nn.functional.cosine_similarity(gtr_emb, support_set_gtr, dim=-1)
    dpr_emb = torch.tensor(dpr.encode([t]))
    support_set_scores = torch.nn.functional.cosine_similarity(dpr_emb, support_set_dpr, dim=-1)
    support_set_dist = (support_set_scores / temp).softmax(0)
    gtr_emb = (support_set_dist[None] @ support_set_gtr)
    #### import pdb; pdb.set_trace()
    print(support_set_dist.max())
    gtr_emb = gtr_emb.to(corr_trainer.args.device)
    # invert
    fake_embedder_input_ids = torch.ones((1, 32), device=corr_trainer.args.device)
    fake_embedder_attention_mask = torch.ones((1, 32), device=corr_trainer.args.device)

    corr_trainer.inversion_trainer.model.use_frozen_embeddings_as_input = True
    preds_sample = corr_trainer.inversion_trainer.generate(
        inputs={ "frozen_embeddings": gtr_emb, "embedder_input_ids": fake_embedder_input_ids, "embedder_attention_mask": fake_embedder_attention_mask },
        generation_kwargs = {'early_stopping': False,  'num_beams': 1,  'do_sample': False,  'no_repeat_ngram_size': 0, 'max_length': 32 },
    )
    
    return et.batch_decode(preds_sample, skip_special_tokens=True)

print(text)
print(invert_with_support(text))

Mage (foaled April 18, 2020) is my American Thoroughbred racehorse who won the 2023 Kentucky Derby.
tensor(0.0643)
['The 2018 Annual Forecast will be available for you to make a profit from your own Investing in the future. Most credit cards have minimum 10 percent']
